# Recreating the Nile result

In [ ]:
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import numpy as np
import os
import scipy
import sys
%matplotlib inline

sys.path.append('..')

from cp_probability_model import CpModel
from BVAR_NIG import BVARNIG
from detector import Detector
from Evaluation_tool import EvaluationTool
from nile_ICML18 import load_nile_data

### Read in the data

In [ ]:
T, S1, S2, river_height, __, dates = load_nile_data(os.path.join(os.getcwd(), "..", "Data", "nile.txt"))

### Set up initial hyperparameters and lag lengths

These values will be updated and optimised as we go along.

In [ ]:
intensity = 100
cp_model = CpModel(intensity)
a, b = 1, 1
prior_mean_scale, prior_var_scale = 0, 0.0075

### Set up the autoregression models

For a set of initial hyperparameters across a range of lag times, generate the model objects.

In [ ]:
upper_AR = 3
lower_AR = 1
AR_models = []

for lag in range(lower_AR, upper_AR + 1):
    """Generate next model object"""
    AR_models += [BVARNIG(
        prior_a=a, prior_b=b,
        S1=S1, S2=S2,
        prior_mean_scale=prior_mean_scale,
        prior_var_scale=prior_var_scale,
        intercept_grouping=None,
        nbh_sequence=[0] * lag,
        restriction_sequence=[0] * lag,
        hyperparameter_optimization="online")]


### Create the model universe 

Put all the model objects together, create model universe and the model priors.


In [ ]:
model_universe = np.array(AR_models)
model_prior = np.array([1 / len(model_universe)] * len(model_universe))

### Build and run the detector

In [ ]:
detector = Detector(
    data=river_height,
    model_universe=model_universe,
    model_prior=model_prior,
    cp_model=cp_model,
    S1=S1, S2=S2, T=T,
    store_rl=True, store_mrl=True,
    trim_type="keep_K", threshold=50,
    notifications=50,
    save_performance_indicators=True)
detector.run()

### Generate results for Table 1

In [ ]:
print("MSE is %.3g with 95%% error of %.3g" %
      (np.mean(detector.MSE), 1.96 * scipy.stats.sem(detector.MSE)))
print("NLL is %.3g with 95%% error of %.3g" %
      (np.mean(detector.negative_log_likelihood), 1.96 * scipy.stats.sem(detector.negative_log_likelihood)))

### Prepare Figure 5

In [ ]:
# We'll prepare the figure using the functionality available via the EvaluationTool class
EvT = EvaluationTool()
EvT.build_EvaluationTool_via_run_detector(detector)

In [ ]:
# Prepare the axes
fig, ax_array = plt.subplots(2, figsize=(8, 5), sharex=True,
                             gridspec_kw={'height_ratios': [10, 14]})
plt.subplots_adjust(hspace=.35, left=None, bottom=None, right=None, top=None)

# Placement of y-labels
ylabel_coords = [-0.065, 0.5]

# Upper panel: plot the time-series
EvT.plot_raw_TS(river_height[2:].reshape(T-2, 1),
                indices=[0],
                xlab=None,
                show_MAP_CPs=True,
                time_range=np.linspace(1, T-2, T-2, dtype=int),
                print_plt=False,
                ylab="River Height", ax=ax_array[0],
                all_dates=np.linspace(int(dates[1]), int(dates[-2]), T-2, dtype=int),
                custom_colors_series=["black"],
                custom_colors_CPs=["blue", "blue"],
                custom_linestyles=["solid"] * 2,
                ylab_fontsize=14,
                ylabel_coords=ylabel_coords,
                set_ylims=(-2.75, 3.95))

# Lower panel: plot the run length distribution
EvT.plot_run_length_distr(buffer=0,
                          show_MAP_CPs=True,
                          mark_median=False,
                          mark_max=True,
                          upper_limit=660,
                          print_colorbar=True,
                          colorbar_location='bottom',
                          log_format=True,
                          aspect_ratio='auto',
                          C1=0, C2=1,
                          time_range=np.linspace(1, T-2, T-2, dtype=int),
                          start=int(dates[2]), stop=int(dates[-1]),
                          event_time_list=[715],    # Nilometer installed in year 715
                          label_list=["nilometer"],
                          space_to_colorbar=0.52,
                          custom_colors=["blue", "blue"],
                          custom_linestyles=["solid"] * 3,
                          custom_linewidth=3,
                          arrow_colors=["black"],
                          number_fontsize=14,
                          arrow_length=135,
                          arrow_thickness=3.0,
                          xlab_fontsize=14,
                          ylab_fontsize=14,
                          arrows_setleft_indices=[0],
                          arrows_setleft_by=[50],
                          zero_distance=0.0,
                          ax=ax_array[1], figure=fig,
                          no_transform=True,
                          date_instructions_formatter=None,
                          date_instructions_locator=None,
                          ylabel_coords=ylabel_coords,
                          xlab="Year",
                          arrow_distance=25)